In [2]:
import sh
from toolz import thread_first, compose
from operator import getitem

In [3]:
server = sh.ssh.bake('-i','../my-pair.pem','ubuntu@52.8.164.123')

In [4]:
def first(x):
    return x[0]

In [13]:
def get_matching_processes(pattern,m=sh):
    all_processes = m.ps('-A')
    target_processes = m.grep(all_processes,pattern, _ok_code = [0,1]).split('\n')[:-1]
    target_pids = map(compose(int,first,str.split,str),target_processes)
    return target_pids

In [14]:
def kill_pids(pids,m=sh):
    for pid in pids:
        m.kill(pid)

In [15]:
def kill_matching_processes(pattern,m=sh):
    pids = get_matching_processes(pattern,m)
    kill_pids(pids,m=sh)

In [16]:
#kill_matching_processes("ipython",server)

In [17]:
m = sh
path = "/home/carlos/projects/sens/data analysis"
m.cd(path)
git fetch --all
git reset --hard origin/master
#m.git('fetch','--all')
#m.git('reset','.')

# On branch master
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   test/.ipynb_checkpoints/Main-checkpoint.ipynb
#	modified:   test/Main.ipynb
#
# Untracked files:
#   (use "git add <file>..." to include in what will be committed)
#
#	test/.ipynb_checkpoints/Deploy-Copy1-checkpoint.ipynb
#	test/.ipynb_checkpoints/Deploy-checkpoint.ipynb
#	test/Deploy-Copy1.ipynb
#	test/Deploy.ipynb
no changes added to commit (use "git add" and/or "git commit -a")